In [129]:
#!pip install -U lime

import pandas as pd
import numpy  as np

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tabulate import tabulate
# from texttable import Texttable
from sklearn.model_selection import train_test_split

from plotly.subplots import make_subplots

from collections import Counter
from sklearn.datasets import make_classification
from imblearn.under_sampling import RandomUnderSampler

In [130]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from sklearn import metrics
from scipy.stats import ks_2samp
import matplotlib.pyplot as plt

In [131]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import statsmodels.api as sm

In [132]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [133]:
import warnings
warnings.filterwarnings('ignore')

In [134]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [135]:
base1 = pd.read_csv("/content/drive/MyDrive/Curso_KU/Projeto_MLops/AmostraBehavior_v2.csv", sep = ";")
# base1 = base1.sample(n = 5000, random_state = 1)
base1.head()

,Id,Target,UltPercLimit,Idade,N_Atraso30_59Dias,RazaoGastos,RendaMensal,N_EmeprestimosAbertos,N_atrasos_Ult90Dias,N_emprestimos,...,CAT_N_EmeprestimosAbertos,CAT_lnRazaoGastos,WOE_N_atrasos_Ult90Dias,WOE_N_emprestimos,WOE_N_Atraso60_89Dias,WOE_N_dependentes,WOE_CAT_UltPercLimit,WOE_CAT_Idade,WOE_CAT_lnRazaoGastos,WOE_CAT_N_EmeprestimosAbertos
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,4,...,5,4,0.389724,-0.592980,0.288208,-0.215545,-1.242254,-0.246922,-0.425358,0.002550
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,5,...,1,1,0.389724,-0.235970,0.288208,-0.108816,-1.242254,-0.246922,0.097667,-0.349083
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,5,...,1,1,-1.957968,-0.235970,0.288208,0.133590,-0.053164,-0.496593,0.097667,-0.349083
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,5,...,2,1,0.389724,-0.235970,0.288208,0.133590,0.866502,-0.496593,0.097667,0.128496
4,5,0,0.907239,49,1,0.024926,25000.0,7,0,1,...,3,1,0.389724,0.256641,0.288208,0.133590,-1.242254,-0.072734,0.097667,0.221596


In [136]:
# prompt: imputar a média da colunas onde tiver calores nulos

def imputar_media_colunas(df):
  for col in df.columns:
    if df[col].dtype in ['int64', 'float64']: # Verifica se a coluna é numérica
      if df[col].isnull().any(): # Verifica se há valores nulos na coluna
        media = df[col].mean()
        df[col] = df[col].fillna(media)
  return df

# Exemplo de uso:
base1 = imputar_media_colunas(base1)
base1.head() # Mostra as primeiras linhas do DataFrame após a imputação


,Id,Target,UltPercLimit,Idade,N_Atraso30_59Dias,RazaoGastos,RendaMensal,N_EmeprestimosAbertos,N_atrasos_Ult90Dias,N_emprestimos,...,CAT_N_EmeprestimosAbertos,CAT_lnRazaoGastos,WOE_N_atrasos_Ult90Dias,WOE_N_emprestimos,WOE_N_Atraso60_89Dias,WOE_N_dependentes,WOE_CAT_UltPercLimit,WOE_CAT_Idade,WOE_CAT_lnRazaoGastos,WOE_CAT_N_EmeprestimosAbertos
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,4,...,5,4,0.389724,-0.592980,0.288208,-0.215545,-1.242254,-0.246922,-0.425358,0.002550
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,5,...,1,1,0.389724,-0.235970,0.288208,-0.108816,-1.242254,-0.246922,0.097667,-0.349083
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,5,...,1,1,-1.957968,-0.235970,0.288208,0.133590,-0.053164,-0.496593,0.097667,-0.349083
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,5,...,2,1,0.389724,-0.235970,0.288208,0.133590,0.866502,-0.496593,0.097667,0.128496
4,5,0,0.907239,49,1,0.024926,25000.0,7,0,1,...,3,1,0.389724,0.256641,0.288208,0.133590,-1.242254,-0.072734,0.097667,0.221596


In [137]:
base1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 26 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Id                             150000 non-null  int64  
 1   Target                         150000 non-null  int64  
 2   UltPercLimit                   150000 non-null  float64
 3   Idade                          150000 non-null  int64  
 4   N_Atraso30_59Dias              150000 non-null  int64  
 5   RazaoGastos                    150000 non-null  float64
 6   RendaMensal                    150000 non-null  float64
 7   N_EmeprestimosAbertos          150000 non-null  int64  
 8   N_atrasos_Ult90Dias            150000 non-null  int64  
 9   N_emprestimos                  150000 non-null  int64  
 10  N_Atraso60_89Dias              150000 non-null  int64  
 11  N_dependentes                  150000 non-null  float64
 12  lnRazaoGastos                 

---

# Preparação ABT modelagem

---

In [138]:
labels = base1.columns[2:len(base1.columns)]

X = base1[labels] # Covariáveis
y = base1.Target # Target

# Base dados balanceados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 999)
X_val, X_test, y_val, y_test     = train_test_split(X_test, y_test, test_size = 0.5, random_state = 999)

print(y_train.count(), y_test.count(),y_val.count())

90000 30000 30000


---

# Balanceamento da informação

---

In [139]:
# https://machinelearningmastery.com/random-oversampling-and-undersampling-for-imbalanced-classification/

under = RandomUnderSampler(sampling_strategy=0.5)
X_under_train, y_under_train = under.fit_resample(X_train, y_train)
tab2  = pd.crosstab(index = y_under_train, columns = ["count"])
tab2["percent"] = (tab2/tab2.sum())*100
tab2

col_0,count,percent
Target,,
0,12082,66.666667
1,6041,33.333333


In [140]:
# Comparando as respostas

tab_under  = pd.crosstab(index = y_under_train, columns = ["count"])
tab_under["percent_under"] = (tab_under/tab_under.sum())*100
tab_under = tab_under.reset_index()

tab_orig  = pd.crosstab(index = y_train, columns = ["count"])
tab_orig["percent_orig"] = (tab_orig/tab_orig.sum())*100
tab_orig = tab_orig.reset_index()


fig_comp = make_subplots(rows=1, cols=2)

trace0 = go.Bar(x = ["Bom","Mau"], y = tab_under["percent_under"], name = "Balanceado", marker_color = "#0066cc")
trace1 = go.Bar(x = ["Bom","Mau"], y = tab_orig["percent_orig"], name = "DesBalanceado", marker_color = "#0099cc")

fig_comp.append_trace(trace0,  1, 1)
fig_comp.append_trace(trace1,  1, 2)

fig_comp.update_layout(height = 400, width = 900, title = "Comparação entre target balanceado e desbalanceado")

fig_comp.show()


In [141]:
X_under_train.describe()

,UltPercLimit,Idade,N_Atraso30_59Dias,RazaoGastos,RendaMensal,N_EmeprestimosAbertos,N_atrasos_Ult90Dias,N_emprestimos,N_Atraso60_89Dias,N_dependentes,...,CAT_N_EmeprestimosAbertos,CAT_lnRazaoGastos,WOE_N_atrasos_Ult90Dias,WOE_N_emprestimos,WOE_N_Atraso60_89Dias,WOE_N_dependentes,WOE_CAT_UltPercLimit,WOE_CAT_Idade,WOE_CAT_lnRazaoGastos,WOE_CAT_N_EmeprestimosAbertos
count,18123.000000,18123.000000,18123.000000,18123.000000,18123.000000,18123.000000,18123.000000,18123.000000,18123.000000,18123.000000,...,18123.000000,18123.000000,18123.000000,18123.000000,18123.000000,18123.000000,18123.000000,18123.000000,18123.000000,18123.000000
mean,0.428535,50.383380,0.456712,305.614338,6135.526969,8.192132,0.235281,2.984936,0.154665,0.805396,...,2.848259,3.034818,0.007700,0.008500,0.005768,0.003582,0.170873,0.048307,0.009153,0.006682
std,0.391016,14.411952,0.935946,898.248635,3804.877953,5.060570,0.663703,1.772469,0.455471,1.096791,...,1.428244,1.401062,0.969234,0.247539,0.792382,0.158792,1.080367,0.529231,0.239968,0.217550
min,0.000000,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,1.000000,1.000000,-3.060716,-0.592980,-2.757084,-0.307485,-1.242254,-0.496593,-0.425358,-0.349083
25%,0.051181,40.000000,0.000000,0.181602,3650.000000,4.000000,0.000000,1.000000,0.000000,0.000000,...,1.000000,2.000000,0.389724,-0.235970,0.288208,-0.108816,-1.242254,-0.246922,0.084360,-0.349083
50%,0.305032,50.000000,0.000000,0.385123,6166.000000,7.000000,0.000000,2.000000,0.000000,0.000000,...,3.000000,3.000000,0.389724,0.188639,0.288208,0.133590,-0.053164,-0.072734,0.097667,0.122389
75%,0.845665,61.000000,1.000000,0.886161,7000.000000,11.000000,0.000000,5.000000,0.000000,1.000000,...,4.000000,4.000000,0.389724,0.256641,0.288208,0.133590,1.286617,0.432726,0.166025,0.128496
max,1.092956,87.000000,4.000000,4979.040000,25000.000000,24.000000,3.000000,5.000000,2.000000,4.000000,...,5.000000,5.000000,0.389724,0.256641,0.288208,0.133590,1.447382,1.063738,0.201802,0.221596


In [142]:
X_train.describe()

,UltPercLimit,Idade,N_Atraso30_59Dias,RazaoGastos,RendaMensal,N_EmeprestimosAbertos,N_atrasos_Ult90Dias,N_emprestimos,N_Atraso60_89Dias,N_dependentes,...,CAT_N_EmeprestimosAbertos,CAT_lnRazaoGastos,WOE_N_atrasos_Ult90Dias,WOE_N_emprestimos,WOE_N_Atraso60_89Dias,WOE_N_dependentes,WOE_CAT_UltPercLimit,WOE_CAT_Idade,WOE_CAT_lnRazaoGastos,WOE_CAT_N_EmeprestimosAbertos
count,90000.000000,90000.000000,90000.000000,90000.000000,90000.000000,90000.000000,90000.000000,90000.000000,90000.000000,90000.000000,...,90000.000000,90000.000000,90000.000000,90000.000000,90000.000000,90000.000000,90000.000000,90000.000000,90000.000000,90000.000000
mean,0.321724,52.293422,0.245733,318.850884,6345.313298,8.389022,0.086656,2.867300,0.063556,0.744481,...,2.906100,3.002489,0.240755,0.026046,0.168344,0.011930,0.457803,0.119455,0.024624,0.019537
std,0.352742,14.675079,0.668429,909.232600,3885.498694,4.937471,0.402207,1.764059,0.291062,1.062563,...,1.404886,1.415688,0.622924,0.243296,0.525008,0.155902,0.999485,0.551809,0.229626,0.211307
min,0.000000,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,1.000000,1.000000,-3.060716,-0.592980,-2.757084,-0.307485,-1.242254,-0.496593,-0.425358,-0.349083
25%,0.030259,41.000000,0.000000,0.175018,3922.000000,5.000000,0.000000,1.000000,0.000000,0.000000,...,2.000000,2.000000,0.389724,-0.235970,0.288208,-0.108816,-0.053164,-0.246922,0.084360,0.002550
50%,0.155774,52.000000,0.000000,0.366799,6349.112332,8.000000,0.000000,2.000000,0.000000,0.000000,...,3.000000,3.000000,0.389724,0.188639,0.288208,0.133590,0.866502,-0.072734,0.097667,0.122389
75%,0.563140,63.000000,0.000000,0.878100,7373.250000,11.000000,0.000000,5.000000,0.000000,1.000000,...,4.000000,4.000000,0.389724,0.256641,0.288208,0.133590,1.286617,0.432726,0.166025,0.128496
max,1.092956,87.000000,4.000000,4979.040000,25000.000000,24.000000,3.000000,5.000000,2.000000,4.000000,...,5.000000,5.000000,0.389724,0.256641,0.288208,0.133590,1.447382,1.063738,0.201802,0.221596


---

# Funções de Avaliação dos modelos

---

In [143]:
# Função para diagnóstico dos modelos

def AVALIA_MOD(modelo, y_prob, y_obs, titulo, base):

    pred_Y_test = {}
    pred_Y_data = pd.DataFrame(modelo.predict_proba(y_prob)[:,1], columns=["y_prob"])
    pred_Y_data["y_hat"] = modelo.predict(y_prob)
    obs_Y_data = y_obs.reset_index()
    pred_data = pd.concat([obs_Y_data,
                         pred_Y_data], axis = 1)

    pred_data["erro2"]     = (pred_data.Target - pred_data.y_prob)**2

    print("--------------------------------------------")
    print("----------------",base,"--------------------")
    print("\n")
    KS = ks_2samp(pred_data.loc[pred_data.Target == 0, "y_prob"], pred_data.loc[pred_data.Target == 1, "y_prob"])
    KS = round(KS[0],2)*100

    ASE = round(np.mean(pred_data.erro2),2)

    AUC = roc_auc_score(pred_data.Target,pred_data.y_prob)
    GINI = 2*AUC - 1
    AUC = round(AUC, 2)*100

    # Curva Roc
    fpr, tpr, thresholds  = roc_curve(pred_data.Target,pred_data.y_prob)

    trace1 = go.Scatter(x = fpr, y = tpr,
                        mode = "lines",
                        line = dict(color = "#b30000", width = 2),
                        name = 'AUC: %0.2f' % AUC)
    trace2 = go.Scatter(x=[0, 1], y=[0, 1],
                        mode='lines',
                        line=dict(color = "navy", width=2, dash = "dash"),
                        showlegend=False)
    layout = go.Layout(xaxis=dict(title = "FP"),
                       yaxis=dict(title = "TP"))
    plot_roc = go.Figure(data=[trace1, trace2], layout=layout)
    plot_roc.update_layout(title_text = "Curva ROC "+titulo)

    # Matriz de confusão
    cm = confusion_matrix(pred_data.Target, pred_data.y_hat)
    conf_m = cm/np.sum(cm)*100
    TP = round(conf_m[0,0],2)
    FP = round(conf_m[0,1],2)
    FN = round(conf_m[1,0],2)
    TN = round(conf_m[1,1],2)

    print("Matriz de confusão default",
         "\n","|","TP", TP,"|","FP",FP,"|","\n","|","FN", FN,"|","TN", TN,"|")
    print("\n")

    CONCORD = round(TP + TN ,2)
    PRECS   = TP/(FP+TP)
    RECALL  = TP/(FN+TP)

    LogLoss = log_loss(pred_data.Target, pred_data.y_hat)

    d = {"Metricas": ["Acurácia","Precisão","Recall","LogLoss","AUC","KS","Gini","ASE"],
         "Valores": [CONCORD,PRECS,RECALL,LogLoss,AUC, KS,GINI,ASE]}

    # Base de medidas de qualidade dos ajuste
    METRICAS = pd.DataFrame(data=d)

    # Base para gráfico de ordenação
    pred_data = pred_data.sort_values(by=["y_prob"])

    pred_data["FX_yprob"] = pd.qcut(pred_data.y_prob, 10)
    pred_data["Default"]  = np.mean(pred_data.Target)

    ordena = pred_data.groupby("FX_yprob")[["Target","Default"]].mean().reset_index()
    ordena["FX_yprob2"] = lab = np.arange(10)
    ordena["LIFT"] = ordena.Target/ordena.Default

#--------------------------------------------------------
    lab1 = titulo[7:]

    if lab1 == "Desbalanc":
        col = "#008ae6"
    else:
        col = "#8a8a5c"
#----------------------------------------------------------
    # Gráfico de ordenação
    fig_ord = data=go.Scatter(x = ordena.FX_yprob2, y = ordena.LIFT, name = "Lift "+titulo, marker=dict(color = col))
#    fig_ord.update_layout(title_text = "Ordenação: Lift "+titulo)

    print("Acurácia:", CONCORD,"|","Precisão:", round(PRECS,2),"|","Recall:", round(RECALL,2))
    print("\n")
    print("KS:", KS,"|", "AUC:",AUC,"|","GINI:",round(GINI,2), "|","LogLoss:",round(LogLoss,2), "|","ASE:", ASE)
    print("\n")
    return(pred_data,ordena,METRICAS,fig_ord,plot_roc,titulo,lab1,AUC)

In [144]:
# Função para avaliar importância das variáveis/features
def ML_IMPORTANCIA(modelo, feat):
    importances = modelo.feature_importances_
    indices = np.argsort(importances)
    indices = pd.Series(indices)
    features = pd.Series(feat)

    print("    Results: Importância das Variáveis")
    print("============================================")
    tt = pd.Series(importances[indices],
                 index=features[indices]).sort_values(ascending=False)
    print(tt)
    print("============================================")

---

# Regressão Logística

---

In [145]:
base1.columns

Index(['Id', 'Target', 'UltPercLimit', 'Idade', 'N_Atraso30_59Dias',
       'RazaoGastos', 'RendaMensal', 'N_EmeprestimosAbertos',
       'N_atrasos_Ult90Dias', 'N_emprestimos', 'N_Atraso60_89Dias',
       'N_dependentes', 'lnRazaoGastos', 'CAT_UltPercLimit', 'CAT_Idade',
       'CAT_RazaoGastos', 'CAT_N_EmeprestimosAbertos', 'CAT_lnRazaoGastos',
       'WOE_N_atrasos_Ult90Dias', 'WOE_N_emprestimos', 'WOE_N_Atraso60_89Dias',
       'WOE_N_dependentes', 'WOE_CAT_UltPercLimit', 'WOE_CAT_Idade',
       'WOE_CAT_lnRazaoGastos', 'WOE_CAT_N_EmeprestimosAbertos'],
      dtype='object')

-----

#  Logit: Balanceados

-----

In [146]:
features = ['UltPercLimit', 'Idade', 'N_Atraso30_59Dias',
       'RendaMensal', 'N_EmeprestimosAbertos',
       'N_atrasos_Ult90Dias', 'N_emprestimos', 'N_Atraso60_89Dias',
       'N_dependentes', 'lnRazaoGastos']

In [147]:
# Modelo Balanceado covariáveis continuas

M_logit_under = LogisticRegression(penalty=None,
                                    solver='lbfgs',
                                    fit_intercept=True,
                                    max_iter = 1000).fit(X_under_train[features], y_under_train)


# Modelo  balanceado com Contínuas
logit_under_test = AVALIA_MOD(modelo = M_logit_under, y_prob = X_test[features], y_obs = y_test, base="Teste", titulo = "Logit: Balanc")
logit_under_val   = AVALIA_MOD(modelo = M_logit_under, y_prob = X_val[features], y_obs = y_val, base="Valida", titulo = "Logit: Balanc")


--------------------------------------------
---------------- Teste --------------------


Matriz de confusão default 
 | TP 84.32 | FP 8.93 | 
 | FN 2.74 | TN 4.01 |


Acurácia: 88.33 | Precisão: 0.9 | Recall: 0.97


KS: 54.0 | AUC: 85.0 | GINI: 0.7 | LogLoss: 4.21 | ASE: 0.09


--------------------------------------------
---------------- Valida --------------------


Matriz de confusão default 
 | TP 84.43 | FP 9.04 | 
 | FN 2.76 | TN 3.78 |


Acurácia: 88.21 | Precisão: 0.9 | Recall: 0.97


KS: 55.00000000000001 | AUC: 85.0 | GINI: 0.7 | LogLoss: 4.25 | ASE: 0.09




----

# Logit: Não Balanceados

----

In [148]:
# Modelo Não Balanceado covariáveis continuas

M_logit = LogisticRegression().fit(X_train[features], y_train)

# Modelo balanceado com Contínuas
pred_Y_test = {}
logit_test = AVALIA_MOD(modelo = M_logit, y_prob = X_test[features], y_obs = y_test, titulo = "Logit: Desbalanc", base="Teste")
logit_val  = AVALIA_MOD(modelo = M_logit, y_prob = X_val[features], y_obs = y_val, titulo = "Logit: Desbalanc", base="Treino")

--------------------------------------------
---------------- Teste --------------------


Matriz de confusão default 
 | TP 92.48 | FP 0.77 | 
 | FN 5.82 | TN 0.93 |


Acurácia: 93.41 | Precisão: 0.99 | Recall: 0.94


KS: 44.0 | AUC: 79.0 | GINI: 0.59 | LogLoss: 2.38 | ASE: 0.05


--------------------------------------------
---------------- Treino --------------------


Matriz de confusão default 
 | TP 92.7 | FP 0.77 | 
 | FN 5.56 | TN 0.97 |


Acurácia: 93.67 | Precisão: 0.99 | Recall: 0.94


KS: 43.0 | AUC: 79.0 | GINI: 0.58 | LogLoss: 2.28 | ASE: 0.05




---

# Random Forest

---

<center><strong><font color = "darkblue" size=4> -----------------  RF: Seleção de Hiperparâmetros -------------------</font></strong></center>

In [149]:
n_estimators = [int(x) for x in np.linspace(start=50, stop=400, num=8)]
max_features = ['sqrt', 'log2']
max_depth = [int(x) for x in np.linspace(2, 15, num=8)]
max_depth.append(None)
bootstrap = [True]

random_grid = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'bootstrap': bootstrap
}

random_grid


{'n_estimators': [50, 100, 150, 200, 250, 300, 350, 400],
 'max_features': ['sqrt', 'log2'],
 'max_depth': [2, 3, 5, 7, 9, 11, 13, 15, None],
 'bootstrap': [True]}

----

# RF: Dados contínuos Não balanceados

----

In [150]:
# Criando o modelo base
rf = RandomForestClassifier()
# Utilizando 3 fold cross validation
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100,
                               cv = 3, verbose=2, random_state=999, n_jobs = -1)

In [151]:
rf_random.fit(X_under_train[features], y_under_train)
rf_random.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


{'n_estimators': 150,
 'max_features': 'log2',
 'max_depth': 9,
 'bootstrap': True}

In [152]:
rf = RandomForestClassifier(**rf_random.best_params_).fit(X_train[features], y_train)

ML_IMPORTANCIA(rf, features)

# Modelo Não balanceado com Contínuas
rf_test = AVALIA_MOD(modelo = rf, y_prob = X_test[features], y_obs = y_test, titulo = "Randf: Desbalanc", base="Teste")
rf_val  = AVALIA_MOD(modelo = rf, y_prob = X_val[features],  y_obs = y_val,  titulo = "Randf: Desbalanc",  base="Valida")

    Results: Importância das Variáveis
N_atrasos_Ult90Dias      0.305772
UltPercLimit             0.190059
N_Atraso60_89Dias        0.142762
N_Atraso30_59Dias        0.131210
lnRazaoGastos            0.060216
Idade                    0.054147
RendaMensal              0.047434
N_EmeprestimosAbertos    0.038838
N_dependentes            0.015939
N_emprestimos            0.013621
dtype: float64
--------------------------------------------
---------------- Teste --------------------


Matriz de confusão default 
 | TP 92.47 | FP 0.78 | 
 | FN 5.64 | TN 1.11 |


Acurácia: 93.58 | Precisão: 0.99 | Recall: 0.94


KS: 55.00000000000001 | AUC: 86.0 | GINI: 0.71 | LogLoss: 2.31 | ASE: 0.05


--------------------------------------------
---------------- Valida --------------------


Matriz de confusão default 
 | TP 92.71 | FP 0.75 | 
 | FN 5.46 | TN 1.07 |


Acurácia: 93.78 | Precisão: 0.99 | Recall: 0.94


KS: 56.00000000000001 | AUC: 86.0 | GINI: 0.71 | LogLoss: 2.24 | ASE: 0.05




----

# RF: Dados contínuos balanceados

----

In [153]:
# Utilizando 3 fold cross validation
rf_random_under = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100,
                               cv = 3, verbose=2, random_state=999, n_jobs = -1)

In [ ]:
rf_random_under.fit(X_under_train[features], y_under_train)
rf_random_under.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


{'n_estimators': 400,
 'max_features': 'log2',
 'max_depth': 9,
 'bootstrap': True}

In [ ]:
rf_under = RandomForestClassifier(**rf_random_under.best_params_).fit(X_under_train[features], y_under_train)

ML_IMPORTANCIA(rf_under, features)

# Modelo Não balanceado com Contínuas
rf_test_under = AVALIA_MOD(modelo = rf_under, y_prob = X_test[features], y_obs = y_test, titulo = "Randf: Balanc", base="Teste")
rf_val_under  = AVALIA_MOD(modelo = rf_under, y_prob = X_val[features],  y_obs = y_val,  titulo = "Randf: Balanc", base="Valida")

    Results: Importância das Variáveis
UltPercLimit             0.296624
N_atrasos_Ult90Dias      0.232792
N_Atraso30_59Dias        0.167823
N_Atraso60_89Dias        0.108539
Idade                    0.057231
lnRazaoGastos            0.044113
RendaMensal              0.034696
N_EmeprestimosAbertos    0.034423
N_emprestimos            0.013407
N_dependentes            0.010352
dtype: float64
--------------------------------------------
---------------- Teste --------------------


Matriz de confusão default 
 | TP 84.33 | FP 8.92 | 
 | FN 2.64 | TN 4.11 |


Acurácia: 88.44 | Precisão: 0.9 | Recall: 0.97


KS: 55.00000000000001 | AUC: 86.0 | GINI: 0.71 | LogLoss: 4.17 | ASE: 0.09


--------------------------------------------
---------------- Valida --------------------


Matriz de confusão default 
 | TP 84.47 | FP 8.99 | 
 | FN 2.65 | TN 3.88 |


Acurácia: 88.35 | Precisão: 0.9 | Recall: 0.97


KS: 56.00000000000001 | AUC: 86.0 | GINI: 0.71 | LogLoss: 4.2 | ASE: 0.09




---

<center><strong><font color = "darkblue" size=4> Comparação dos modelos </font></strong></center>

---

In [ ]:
pdList = [

logit_under_test[2],
logit_test[2].iloc[:,1],

rf_test_under[2].iloc[:,1],
rf_test[2].iloc[:,1],

]

In [ ]:
compara = pd.concat(pdList, axis=1)
compara.columns = ["Metricas", "Logit_balanc", "Logit",
                               "RF_balanc", "RF"]

In [ ]:
compara

,Metricas,Logit_balanc,Logit,RF_balanc,RF
0,Acurácia,88.330000,93.410000,88.440000,93.580000
1,Precisão,0.904236,0.991743,0.904343,0.991635
2,Recall,0.968527,0.940793,0.969645,0.942514
3,LogLoss,4.205093,2.375277,4.169049,2.314003
4,AUC,85.000000,79.000000,86.000000,86.000000
5,KS,54.000000,44.000000,55.000000,55.000000
6,Gini,0.699220,0.587711,0.712956,0.714139
7,ASE,0.090000,0.050000,0.090000,0.050000


In [ ]:
plot_ord = make_subplots(rows = 2, cols = 2,
                        subplot_titles = (
            logit_under_test[5],
            logit_test[5] ,
            rf_test_under[5],
            rf_test[5]
            )
      )


trace11 = logit_under_test[3]
trace12 = logit_test[3]

trace21 = rf_test_under[3]
trace22 = rf_test[3]

plot_ord.append_trace(trace11, 1, 1)
plot_ord.append_trace(trace12, 1, 2)

plot_ord.append_trace(trace21, 2, 1)
plot_ord.append_trace(trace22, 2, 2)

plot_ord.update_layout(height = 800, width = 1000, showlegend = False, title="Ordenação por Lift")
plot_ord.show()

In [ ]:
for j in range(0,8):
    clrs[j] = [clrred if compara.iloc[j,1:][x] == max(compara.iloc[j,1:]) else clrblue for x in range(0, 4)]
    print(clrs[j])

['#00ace6', '#00ace6', '#00ace6', '#73264d']
['#00ace6', '#73264d', '#00ace6', '#00ace6']
['#00ace6', '#00ace6', '#73264d', '#00ace6']
['#73264d', '#00ace6', '#00ace6', '#00ace6']
['#00ace6', '#00ace6', '#73264d', '#73264d']
['#00ace6', '#00ace6', '#73264d', '#73264d']
['#00ace6', '#00ace6', '#00ace6', '#73264d']
['#73264d', '#00ace6', '#73264d', '#00ace6']


In [ ]:
nomes = [compara.iloc[l,0] for l in range(0,8)]
nomes


['Acurácia', 'Precisão', 'Recall', 'LogLoss', 'AUC', 'KS', 'Gini', 'ASE']

In [ ]:
trace = {}
clrs  = {}

clrred  = "#73264d"
clrblue = "#00ace6"

for j in range(0,8):
    clrs[j] = [clrred if compara.iloc[j,1:][x] == max(compara.iloc[j,1:]) else clrblue for x in range(0, 4)]

nomes = [compara.iloc[l,0] for l in range(0,8)]
nomes
fig_comp = make_subplots(rows=8, cols=1, subplot_titles = nomes)
for k in range(0,8):
    trace[k,0] = go.Scatter(x = compara.iloc[0,1:].reset_index().iloc[:,0],
                            y = compara.iloc[k,1:], mode='markers',
                             marker_size = np.repeat(20,20), marker = dict(color = clrs[k]))


    fig_comp.append_trace(trace[k,0], k+1, 1)

fig_comp.update_layout(height = 1500, width = 800, showlegend = False,
                       title="Medidas de qualidade dos modelos")

fig_comp.show()


In [ ]:
# Normalizando os dados usando Min-Max:
compara2 = compara.iloc[[0,3,4,5,6],1:].apply(lambda x: (x - min(x))/(max(x)-min(x)), axis=1)
aux_comp = compara2.apply(np.sum, axis=0).sort_values(ascending=False).reset_index()
aux_comp

fig = go.Figure(data=go.Scatter(x=aux_comp.iloc[:,0], y=aux_comp.iloc[:,1], mode="markers",
                               marker_size = np.repeat(30,20)))

fig.update_layout(height = 400, width = 900, showlegend = False, title="Rank dos modelos")
fig.show()